导入库文件

导入库文件

In [1]:
#导入外部库
import numpy as np
from PIL import Image

#导入自定义文件
from utils import *
from conv import *
from concat import *
from relu import *
# np.set_printoptions(threshold=np.inf)
# np.set_printoptions(precision=32, suppress=False, formatter={'float': '{:0.16}'.format})

加载INT8权重

In [2]:
from weights_int8 import qw1,Sw1,qw2,Sw2,qw3,Sw3,qw4,Sw4,qw5,Sw5
from bias_int8 import qb1,Sb1,qb2,Sb2,qb3,Sb3,qb4,Sb4,qb5,Sb5
from inout_int8 import Sconcat1,Zconcat1,Sconcat2,Zconcat2,Sconcat3,Zconcat3,Smul_out,Zmul_out,Ssub_out,Zsub_out,Sadd_out,Zadd_out
from inout_int8 import Sy1,Zy1,Sy2,Zy2,Sy3,Zy3,Sy4,Zy4,Sy5,Zy5
from inout_int8 import Sx1,Zx1,Sx2,Zx2,Sx3,Zx3,Sx4,Zx4,Sx5,Zx5

参数定义

In [3]:
img_path="./test/img/OTS_2987.jpg"#输入图片
txt_dir = "./test/txt"#txt文件保存路径
txt_input_1 = "./test/txt_dec/conv1_in.txt"#Conv1输入图片   1x1卷积 1*1*3*8 = 24bit
txt_input_2 = "./test/txt_dec/conv2_in.txt"#Conv2输入特征图 3x3卷积 3*3*3*8 = 216bit
txt_input_3 = "./test/txt_dec/conv3_in.txt"#Conv3输入特征图 5x5卷积 5*5*6*8 = 1200bit
txt_input_4 = "./test/txt_dec/conv4_in.txt"#Conv4输入特征图 7x7卷积 7*7*9*8 = 3528bit
txt_input_5 = "./test/txt_dec/conv5_in.txt"#Conv5输入特征图 3x3卷积 3*3*6*8 = 432bit


txt_output_1 = "./test/txt_dec/conv1_out.txt"
txt_output_2 = "./test/txt_dec/conv2_out.txt"
txt_output_3 = "./test/txt_dec/conv3_out.txt"
txt_output_4 = "./test/txt_dec/conv4_out.txt"
txt_output_5 = "./test/txt_dec/conv5_out.txt"

# filter_shape=(output_channel,input_channel,filter_size)
#               输出通道数      输入通道数     卷积核大小
conv1_shape = (3,3,1)
conv2_shape = (3,3,3)
conv3_shape = (3,6,5)
conv4_shape = (3,6,7)
conv5_shape = (3,6,3)
#conv_value=(Kernel_size,stride,padding)
conv1_value = (1,1,0)
conv2_value = (3,1,1)
conv3_value = (5,1,2)
conv4_value = (7,1,3)
conv5_value = (3,1,1)

加载图像

In [4]:
def array_show_int8(array):
    deahze_img = array.astype(np.uint8)
    deahze_img = np.squeeze(deahze_img)
    img = Image.fromarray(deahze_img.transpose(1,2,0))
    img.show()

hazy_input_image = Image.open(img_path)#图像加载
hazy_input_image = hazy_input_image.resize((640, 480))
hazy_image = (np.asarray(hazy_input_image))
hazy_image = hazy_image.transpose(2,0,1)#HWC -> CHW
hazy_image = np.expand_dims(hazy_image,axis=0)#增加一个新的维度batch_size (batch_size,C,H,W)

array_show_int8(hazy_image)

输入图片量化

In [5]:
haze_img_int8 = (hazy_image-128).astype(np.int8)
#Conv1输入
# with open(txt_input_1,"w+") as f:
#     f.write(f"haze_img_int8:{haze_img_int8}\n")

conv-relu1计算

量化卷积计算int8*int8+int32 -> int32

In [6]:
q_conv1 = Q_conv2d_fused(haze_img_int8,qw1,qb1,Sx1,Zx1,Sw1,Sb1,Sy1,Zy1,conv1_shape,stride=1,padding='SAME',const_num=Zx1)

M:[[661 472 494]]
B:[[ -48 -100 -107]]


relu1计算

In [7]:
qrelu1 = Q_ReLU(q_conv1)

In [8]:
# with open(txt_output_1,"w+") as f:
#     f.write(f"haze_img_int8:{qrelu1}\n")

In [9]:
#Conv2输入
# with open(txt_input_2,"w+") as f:
#     f.write(f"qrelu1:{qrelu1}\n")

conv-relu2计算

In [10]:
q_conv2 = Q_conv2d_fused(qrelu1,qw2,qb2,Sx2,Zx2,Sw2,Sb2,Sy2,Zy2,conv2_shape,stride=1,padding='SAME',const_num=Zx2)

M:[[851 335 346]]
B:[[-511 -159  -28]]


In [11]:
qrelu2 = Q_ReLU(q_conv2)
# array_show_int8(qrelu2)
# array_show_int8(qrelu2+128)


In [12]:
# with open(txt_output_2,"w+") as f:
#     f.write(f"haze_img_int8:{qrelu2}\n")

Concat1计算

In [13]:
q_concat1 = requantize(qrelu2,Sy2,Zy2,Sconcat1,Zconcat1)
q_concat1 = concat(qrelu1,q_concat1)

M:[191]


In [14]:
#Conv3输入
# with open(txt_input_3,"w+") as f:
#     f.write(f"q_concat1:{q_concat1}\n")

conv+relu3计算

In [15]:
q_conv3 = Q_conv2d_fused(q_concat1,qw3,qb3,Sx3,Zx3,Sw3,Sb3,Sy3,Zy3,conv3_shape,stride=1,padding='SAME',const_num=Zx3)

M:[[ 741  791 2835]]
B:[[  -418     58 -16276]]


In [16]:
qrelu3 = Q_ReLU(q_conv3)
# array_show_int8(qrelu3)
# array_show_int8(qrelu3+128)

In [17]:
# with open(txt_output_3,"w+") as f:
#     f.write(f"haze_img_int8:{qrelu3}\n")

concat2的计算

In [18]:
q_concat2_int_1 = requantize(qrelu3,Sy3,Zy3,Sconcat2,Zconcat2)
q_concat2 = concat(qrelu2,q_concat2_int_1)

M:[249]


In [19]:
#Conv4输入
# with open(txt_input_4,"w+") as f:
#     f.write(f"q_concat2:{q_concat2}\n")

conv+relu4计算

In [20]:
q_conv4 = Q_conv2d_fused(q_concat2,qw4,qb4,Sx4,Zx4,Sw4,Sb4,Sy4,Zy4,conv4_shape,stride=1,padding='SAME',const_num=Zx4)

M:[[401 124 234]]
B:[[-277  -35 -659]]


In [21]:
qrelu4 = Q_ReLU(q_conv4)
# array_show_int8(qrelu4)
# array_show_int8(qrelu4+128)


In [22]:
# with open(txt_output_4,"w+") as f:
#     f.write(f"haze_img_int8:{qrelu4}\n")

Concat3计算

In [23]:
q_concat3 = requantize(qrelu4,Sy4,Zy4,Sconcat3,Zconcat3)

q_concat3 = concat(qrelu3,q_concat3)
# array_show_int8(q_concat3)
# array_show_int8(q_concat3+128)

M:[666]


In [24]:
#Conv5输入
# with open(txt_input_5,"w+") as f:
#     f.write(f"q_concat3:{q_concat3}\n")

conv+relu5计算

In [25]:
q_conv5 = Q_conv2d_fused(q_concat3,qw5,qb5,Sx5,Zx5,Sw5,Sb5,Sy5,Zy5,conv5_shape,stride=1,padding='SAME',const_num=Zx5)

M:[[62 69 65]]
B:[[166 168 161]]


In [26]:
qrelu5 = Q_ReLU(q_conv5)
# array_show_int8(qrelu5+128)
# array_show_int8(qrelu5)

In [27]:
# with open(txt_output_5,"w+") as f:
#     f.write(f"haze_img_int8:{qrelu5}\n")

mul计算

In [28]:
q_mul = q_mul_fun(qrelu5,Sy5,Zy5,haze_img_int8,Sx1,Zx1,Smul_out,Zmul_out)
print(q_mul)

M:[280]
[[[[-38 -27 -20 ... -13  -3 -27]
   [-28 -16 -16 ...  -2   1  -5]
   [-27 -19 -16 ...  -3  -2 -14]
   ...
   [-42 -36 -28 ... -27 -36 -48]
   [-36 -30 -25 ... -26 -35 -49]
   [-51 -37 -35 ... -36 -46 -53]]

  [[-19   5  10 ...  13  10   5]
   [ -7  14  14 ...  27  26  33]
   [ -3  15  16 ...  22  18  24]
   ...
   [-25  -9  -7 ... -22 -35 -45]
   [-23  -9   0 ... -20 -33 -44]
   [-25 -18 -11 ... -32 -44 -55]]

  [[-12  18  23 ...  28  21  43]
   [  6  28  31 ...  34  32  55]
   [  7  26  26 ...  33  23  48]
   ...
   [-15  -7   0 ... -46 -59 -68]
   [-16   1   2 ... -46 -59 -67]
   [-24  -9  -4 ... -53 -65 -77]]]]


sub计算

In [29]:
q_sub = q_sub_fun(q_mul,Smul_out,Zmul_out,qrelu5,Sy5,Zy5,Ssub_out,Zsub_out)
print(q_sub)


qx1.dtype:int8
qx2.dtype:int8
Sx1/Sy:[1.2318841]
Sx2/Sy:[1.34243776]
M1:[20183]
M2:[-21995]
B:[113]
[[[[ 36  26  18 ...  32  24  43]
   [ 26  14  15 ...  24  21  27]
   [ 26  17  15 ...  24  23  34]
   ...
   [ 18  10   4 ...   3 -14 -12]
   [  9   6   1 ...   3 -13  -7]
   [ 28  13  13 ...  16   5   2]]

  [[ 74  64  62 ...  68  68  70]
   [ 69  60  60 ...  63  63  59]
   [ 67  60  58 ...  63  65  63]
   ...
   [ 48  38  38 ...   5  -4  -5]
   [ 47  41  36 ...   6  -3  -6]
   [ 48  47  43 ...  21  11  15]]

  [[ 96  89  88 ...  93  94  90]
   [ 92  87  86 ...  91  93  88]
   [ 92  87  87 ...  93  94  88]
   ...
   [ 60  58  55 ... -25 -30 -30]
   [ 61  55  56 ... -22 -27 -32]
   [ 67  60  59 ...  -6 -13  -4]]]]


add计算

In [30]:
q_add = q_add_fun(q_sub,Ssub_out,Zsub_out,Sadd_out,Zadd_out)
print(q_add)
q_add_relu = Q_ReLU(q_add)

M:[20488]
C:[-32]
[[[[ 13   0 -10 ...   8  -2  21]
   [  0 -15 -14 ...  -2  -6   1]
   [  0 -11 -14 ...  -2  -4  10]
   ...
   [-10 -20 -27 ... -29 -50 -48]
   [-21 -25 -31 ... -29 -49 -41]
   [  3 -16 -16 ... -12 -26 -30]]

  [[ 60  48  45 ...  53  53  55]
   [ 54  43  43 ...  46  46  41]
   [ 51  43  40 ...  46  49  46]
   ...
   [ 28  15  15 ... -26 -38 -39]
   [ 26  19  13 ... -25 -36 -40]
   [ 28  26  21 ...  -6 -19 -14]]

  [[ 88  79  78 ...  84  85  80]
   [ 83  76  75 ...  81  84  78]
   [ 83  76  76 ...  84  85  78]
   ...
   [ 43  40  36 ... -64 -70 -70]
   [ 44  36  38 ... -60 -66 -73]
   [ 51  43  41 ... -40 -49 -38]]]]


INT8模型输出

In [31]:
q_img_out= q_add_relu+128

array_show_int8(q_img_out)